In [0]:
from pyspark.sql.functions import col,date_diff,current_date

In [0]:
dbutils.widgets.text("business_date", "")
business_date = dbutils.widgets.get("business_date")


In [0]:
today_df = spark.createDataFrame(
    [(business_date,)],
    ["event_date"]
)

In [0]:
# 2. Add late-impacted dates if any
late_dates_df = (
    spark.read.table("silver_sales")
    .filter(col("is_late") == True)
    .select("event_date")
    .distinct()
)


In [0]:
dates_to_process = (
    today_df
    .union(late_dates_df)
    .distinct()
)

In [0]:
gold_df = (
    spark.read.table("silver_sales")
    .join(dates_to_process, "event_date")
    .groupBy("event_date")
    .sum("RETAIL_SALES")
    .withColumnRenamed("sum(RETAIL_SALES)", "Total_RETAIL_SALES")
)

